In [ ]:
# DATASET

In [58]:
import pandas as pd

In [2]:
import sys
print(sys.executable)
print(sys.version)
import site
print(site.getsitepackages())
import os
print(os.environ.get('PATH'))
print(os.environ.get('CUDA_HOME'))

C:\Users\Demensdeum\AppData\Local\Programs\Python\Python310\python.exe
3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]
['C:\\Users\\Demensdeum\\AppData\\Local\\Programs\\Python\\Python310', 'C:\\Users\\Demensdeum\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages']


In [51]:
import json

def flatten_chat_messages_with_specific_columns(input_filename="result.json", output_filename="messages.json"):
    try:
        with open(input_filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: Input file '{input_filename}' not found.")
        return
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{input_filename}'.")
        return
    except Exception as e:
        print(f"An unexpected error occurred while reading '{input_filename}': {e}")
        return

    selected_messages = []
    desired_columns = ["from", "text", "date"]

    if 'chats' in data and isinstance(data['chats'], dict) and \
       'list' in data['chats'] and isinstance(data['chats']['list'], list):
        for chat_entry in data['chats']['list']:
            if not 'name' in chat_entry:
                continue
                
            chat_id = chat_entry['name']
            
            if chat_id == None:
                chat_id = "Unknown"
                
            if 'messages' in chat_entry and isinstance(chat_entry['messages'], list):
                for message in chat_entry['messages']:
                    filtered_message = {}
                    for col in desired_columns:
                        if col in message:
                            filtered_message[col] = message[col]

                    if filtered_message and len(filtered_message["text"]) > 0:
                        filtered_message["chat_id"] = chat_id
                        selected_messages.append(filtered_message)
    else:
        print("Warning: Expected 'chats.list' structure not found or is not in the correct format in your input JSON. No messages will be extracted.")

    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(selected_messages, f, indent=4, ensure_ascii=False)
        print(f"Successfully extracted {len(selected_messages)} messages with specific columns to '{output_filename}'.")
    except IOError as e:
        print(f"Error writing to output file '{output_filename}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred while writing '{output_filename}': {e}")

if __name__ == "__main__":
    flatten_chat_messages_with_specific_columns("result.json", "messages.json")

Successfully extracted 85352 messages with specific columns to 'messages.json'.


In [17]:
df = pd.read_json('messages.json')

In [38]:
#target_chat_id_prefix = "Bars"
#filtered_df = df[df['chat_id'].astype(str).str.startswith(target_chat_id_prefix, na=False)]
#sorted_df = filtered_df.sort_values(by='date')
#print(f"\nMessages for chat_id: '{target_chat_id}' (sorted by date):")

sorted_df = df.sort_values(by='date')
print(sorted_df)

                             from  \
6270  Ilia Prokhorov (DemensDeum)   
6271  Ilia Prokhorov (DemensDeum)   
6272  Ilia Prokhorov (DemensDeum)   
6273               Серега Федоров   
6274               Серега Федоров   
...                           ...   
6269                Игорь Беларус   
6194  Ilia Prokhorov (DemensDeum)   
6195                Mariya Bogach   
6196                Mariya Bogach   
6197  Ilia Prokhorov (DemensDeum)   

                                                   text                date  \
6270                                              Йопта 2016-12-28 01:56:43   
6271                                 Так инсту заведешь 2016-12-28 01:56:48   
6272                     Че за картонка у тебя в руках? 2017-04-14 20:39:26   
6273              Вот ток ты сказал, а его уже выпилили 2017-04-18 20:03:33   
6274  Steve Stephens Dead: 5 Fast Facts You Need to ... 2017-04-18 20:03:40   
...                                                 ...                 ...   
62

In [52]:
def create_ollama_finetune_dataset(sorted_messages_df, source_id, output_finetune_file):
    if sorted_messages_df.empty:
        print("No messages provided to create finetuning dataset.")
        return

    finetune_examples = []
    current_instruction = None

    for chat_id, chat_df in sorted_messages_df.groupby('chat_id'):
        print(f"\nProcessing chat: {chat_id} for finetuning data...")
        current_instruction = ""

        for index, message in chat_df.iterrows():
            sender = message.get('from')
            text = message.get('text')

            if sender == source_id:
                if len(current_instruction) > 0:
                    finetune_examples.append({
                        "input": current_instruction,
                        "output": text
                    })
                    current_instruction = ""
            else:
                current_instruction += str(text)

    try:
        with open(output_finetune_file, 'w', encoding='utf-8') as f:
            for example in finetune_examples:
                f.write(json.dumps(example, ensure_ascii=False) + '\n')
        print(f"\nSuccessfully created {len(finetune_examples)} finetuning examples in '{output_finetune_file}'.")
    except IOError as e:
        print(f"Error writing to output file '{output_finetune_file}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred while writing '{output_finetune_file}': {e}")

create_ollama_finetune_dataset(sorted_df, "Ilia Prokhorov (DemensDeum)", "ollama_finetune_dataset.jsonl")


Processing chat: #самарскийлонгбординг for finetuning data...

Processing chat: .:NeCrOmAnCeR:. for finetuning data...

Processing chat: 0JLQuCDQotGP0L0= for finetuning data...

Processing chat: Aleksandr for finetuning data...

Processing chat: Alex for finetuning data...

Processing chat: Alexey for finetuning data...

Processing chat: AllTrust.me for finetuning data...

Processing chat: Andrew Smirnov for finetuning data...

Processing chat: AntiSwap for finetuning data...

Processing chat: Anton for finetuning data...

Processing chat: Bars for finetuning data...

Processing chat: BonBon for finetuning data...

Processing chat: CPT for finetuning data...

Processing chat: Capsule Hotel Tbilisi for finetuning data...

Processing chat: Cheque for finetuning data...

Processing chat: Credo Bank for finetuning data...

Processing chat: Daniil Polyakov for finetuning data...

Processing chat: Denis for finetuning data...

Processing chat: Diwixis for finetuning data...

Processing chat

In [ ]:
# MODEL TRAIN

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

target_model = "unsloth/tinyllama-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = target_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.5: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 3080 Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.7.5 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [5]:
pip install jsonlines

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [7]:
from datasets import Dataset
import jsonlines
import json

formatted_data = []

with jsonlines.open("ollama_finetune_dataset.jsonl", 'r') as reader:
    for obj in reader:
        output = f"### Input: {obj['input']}\n### Output: {json.dumps(obj['output'])}<|endoftext|>"
        formatted_data.append(output)

dataset = Dataset.from_dict({"text": formatted_data})

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments

default_train_params = {
    "max_seq_length" : max_seq_length,
    "dataset_num_proc" : 2,
    "per_device_train_batch_size" : 2,
    "gradient_accumulation_steps" : 4,
    "warmup_steps" : 10,
    "num_train_epochs" : 3,
    "learning_rate" : 2e-4,
    "fp16" : not torch.cuda.is_bf16_supported(),
    "bf16" : torch.cuda.is_bf16_supported(),
    "logging_steps" : 25,  
    "weight_decay" : 0.01,
    "seed" : 3407,
    "save_total_limit" : 2,
    "dataloader_pin_memory" : False
}

faster_train_params = {
    "max_seq_length" : 512,
    "dataset_num_proc" : 4,
    "per_device_train_batch_size" : 4,
    "gradient_accumulation_steps" : 1,
    "warmup_steps" : 0,
    "num_train_epochs" : 1,
    "learning_rate" : 5e-4,
    "fp16" : not torch.cuda.is_bf16_supported(),
    "bf16" : torch.cuda.is_bf16_supported(),
    "logging_steps" : 100,  
    "weight_decay" : 0.0,
    "seed" : 3407,
    "save_total_limit" : 0,
    "dataloader_pin_memory" : True
}

no_train_params = {
    "max_seq_length" : 8, # Drastically reduce sequence length
    "dataset_num_proc" : max(1, torch.cuda.device_count() * 2), # Maximize data loading processes
    "per_device_train_batch_size" : max(1, torch.cuda.device_count() * 8), # Maximize batch size (pushing VRAM limits)
    "gradient_accumulation_steps" : 1, # No gradient accumulation
    "warmup_steps" : 0, # No warmup
    "num_train_epochs" : 1, # Minimum epochs
    "learning_rate" : 1e-2, # Very high learning rate for fast, but likely unstable, steps
    "fp16" : not torch.cuda.is_bf16_supported(),
    "bf16" : torch.cuda.is_bf16_supported(),
    "logging_steps" : 1, # Log every step for immediate feedback
    "weight_decay" : 0.0, # Disable weight decay (regularization)
    "seed" : 42, # A fixed seed
    "save_total_limit" : 0, # Disable saving checkpoints
    "dataloader_pin_memory" : True # Enable pin memory for faster data transfer
}

train_params = no_train_params

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=train_params["max_seq_length"],
    dataset_num_proc=train_params["dataset_num_proc"],
    args=TrainingArguments(
        per_device_train_batch_size=train_params["per_device_train_batch_size"],
        gradient_accumulation_steps=train_params["gradient_accumulation_steps"],  # Effective batch size = 8
        warmup_steps=train_params["warmup_steps"],
        num_train_epochs=train_params["num_train_epochs"],
        learning_rate=train_params["learning_rate"],
        fp16=train_params["fp16"],
        bf16=train_params["bf16"],
        logging_steps=train_params["logging_steps"],
        optim="adamw_8bit",
        weight_decay=train_params["weight_decay"],
        lr_scheduler_type="linear",
        seed=train_params["seed"],
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=train_params["save_total_limit"],
        dataloader_pin_memory=train_params["dataloader_pin_memory"],
        report_to="none", # Disable Weights & Biases logging
    ),
)

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"]:   0%|          | 0/19076 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,076 | Num Epochs = 1 | Total steps = 2,385
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 12,615,680 of 1,112,664,064 (1.13% trained)


Step,Training Loss
1,1.687600
2,7.000500
3,12.412200
4,23.986400
5,13.328200
6,15.241200
7,12.344800
8,17.177900
9,10.663400
10,9.693800


/mnt/c/Users/Demensdeum/Documents/Sources/Demensdeum/llm-chatbot-finetune/myenv/lib/python3.12/site-packages/peft/utils/other.py:1221: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /unsloth/tinyllama-bnb-4bit/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d14ea585c40>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: 4ecceead-8ac5-4b22-aaac-25624d1755b7)') - silently ignoring the lookup for the file config.json in unsloth/tinyllama-bnb-4bit.
  warnings.warn(
/mnt/c/Users/Demensdeum/Documents/Sources/Demensdeum/llm-chatbot-finetune/myenv/lib/python3.12/site-packages/peft/utils/save_and_load.py:238: UserWarning: Could not find a config file in unsloth/tinyllama-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


In [15]:
trainer.model.save_pretrained("output_model")
trainer.tokenizer.save_pretrained("output_model_tokenizer")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('output_model_tokenizer/tokenizer_config.json',
 'output_model_tokenizer/special_tokens_map.json',
 'output_model_tokenizer/tokenizer.model',
 'output_model_tokenizer/added_tokens.json',
 'output_model_tokenizer/tokenizer.json')

In [21]:
!apt install libcurl4-openssl-dev

Reading package lists... Done
Building dependency tree... 50%

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Building dependency tree... Done
Reading state information... Done
libcurl4-openssl-dev is already the newest version (8.5.0-2ubuntu10.6).
The following package was automatically installed and is no longer required:
  libllvm17t64
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [ ]:
# Do that manually
# cd llama.cpp
# cmake .
# make
# then make copy of bin directory into build, rename ollama-quantize to quantize

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.98 out of 15.53 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 26.74it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at gguf_model into bf16 GGUF format.
The output location will be /mnt/c/Users/Demensdeum/Documents/Sources/Demensdeum/llm-chatbot-finetune/gguf_model/unsloth.BF16.gguf
This might take 3 minutes...
